# 数据导入

In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import matplotlib.pyplot as plt

import os
os.chdir(r'\KD_TP3-29_pre_result')############adjust
data=pd.read_csv('KD_TP15.csv',dtype='str')
data=data.dropna()
data=data[['date','sum_xy']]
data['date']=pd.to_datetime(data['date'])
data['sum_xy']=pd.to_numeric(data['sum_xy'])
data

,date,sum_xy
1,2018-04-02,12.714165
2,2018-04-03,24.386783
3,2018-04-04,29.981423
4,2018-04-05,39.586110
5,2018-04-06,45.130476
...,...,...
698,2020-02-28,6354.487653
699,2020-02-29,6360.627685
700,2020-03-01,6364.429001
701,2020-03-02,6368.530220


# 累计位移预测_data_sum

## 数据筛选

In [2]:
data_sum=data

## 按照prophet格式更改数据名称,data_dis

In [3]:
data_sum=data_sum.rename(columns={"sum_xy":"y", "date":"ds"})

## 训练集dataf

In [4]:
dataf = data_sum[0:len(data_sum)-round(1*7)]

## prophet拟合 训练集dataf

In [79]:
model = Prophet(n_changepoints=55,#default=25
                changepoint_range=0.9,#default=0.8
                yearly_seasonality=True,weekly_seasonality=True,changepoint_prior_scale=0.5,interval_width=0.95)

model.fit(dataf)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


##  预测后7天的数据

In [80]:
future = model.make_future_dataframe(periods = round(1*7), freq = "1D")
forecast = model.predict(future)
forecast.tail(7).T

,695,696,697,698,699,700,701
ds,2020-02-26 00:00:00,2020-02-27 00:00:00,2020-02-28 00:00:00,2020-02-29 00:00:00,2020-03-01 00:00:00,2020-03-02 00:00:00,2020-03-03 00:00:00
trend,6406.73,6411.09,6415.44,6419.8,6424.16,6428.52,6432.88
yhat_lower,6335.06,6338.35,6340.36,6341.54,6341.29,6339.75,6338.3
yhat_upper,6361.25,6364.93,6367.76,6371.89,6378.53,6383.45,6391.43
trend_lower,6406.5,6408.13,6409.23,6409.03,6407.88,6406.05,6404.27
trend_upper,6406.94,6413.12,6420.68,6429.49,6439.28,6447.54,6458.5
additive_terms,-58.2727,-59.9235,-61.2935,-62.6322,-63.9913,-65.7251,-66.6924
additive_terms_lower,-58.2727,-59.9235,-61.2935,-62.6322,-63.9913,-65.7251,-66.6924
additive_terms_upper,-58.2727,-59.9235,-61.2935,-62.6322,-63.9913,-65.7251,-66.6924
weekly,0.17411,-0.000835047,0.0555534,0.0965468,0.075545,-0.355123,-0.0457971


## 计算error

In [85]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
error = mean_absolute_percentage_error(data_sum.y[-round(1*7)*1:].values,
                                       forecast.yhat[-round(1*7)*1:].values)
print(error)

0.04675513777665474


In [88]:
from sklearn import metrics
#metrics.r2_score(data_sum.y[-round(1*7)*1:].values,forecast.yhat[-round(1*7)*1:].values)
metrics.mean_absolute_error(data_sum.y[-round(1*7)*1:].values,forecast.yhat[-round(1*7)*1:].values)

2.9770373746741825

## 数据导出

In [76]:
import os
os.chdir(r'\Prophet')############adjust

forecast_all=forecast.merge(data[['date','sum_xy']],how='left',left_on='ds',right_on='date')
forecast_all.to_csv('prophet_TP15.csv',index=False)